# **Задача мультилейблинга в NLP**

In [ ]:
!pip install gensim==4.3.2

In [ ]:
!pip install pandas==1.5.3

In [ ]:
!pip install numpy==1.24.4
!pip install tensorflow

In [ ]:
!pip install protobuf==3.20.*

In [ ]:
!pip install pymorphy2

In [ ]:
!pip install scikeras

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
import re
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from nltk.stem.snowball import RussianStemmer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, hamming_loss

In [ ]:
from gensim.models import Word2Vec

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path_train = '/content/drive/MyDrive/DL/comp2/train.csv'
path_test = '/content/drive/MyDrive/DL/comp2/test.csv'

In [ ]:
df = pd.read_csv(path_train)
test = pd.read_csv(path_test)

## **1) Проанализировать данные, посмотреть на баланс классов, посмотреть на представителей классов, поизучать текста, сделать выводы. (0.5 балла).**

In [ ]:
df.head()

In [ ]:
print(f"Всего записей: {len(df)}")

In [ ]:
all_text = ' '.join(df['text'])
word_freq = Counter(all_text.split())
most_common = dict(word_freq.most_common(100))
least_common = dict(word_freq.most_common()[:-101:-1])

In [ ]:
def plot_wordcloud(word_freq, title):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(title)
    plt.show()

In [ ]:
plot_wordcloud(most_common, "Самые популярные слова")

Из облака слов видно, что все тексты скорее всего как-то связаны с продажей билетов на разные культурные мероприятия. Так же видно, что очень часто встречаются предлоги, возможно стоит их убрать.

In [ ]:
print(df[['text', 'labels']].sample(5))

Много непонятных разделителей \n, капслок, английский и смайлики

In [ ]:
plot_wordcloud(least_common, "Некоторые наименее популярные слова")

Разделяю лейблы:

In [ ]:
labels = df['labels'].str.split(' ', expand=True)
labels = labels.astype(int)
num_classes = labels.shape[1]
labels.columns = [f'label_{i}' for i in range(num_classes)]

In [ ]:
df_train = pd.concat([df, labels], axis=1)

In [ ]:
df_train.head(3)

In [ ]:
all_text = ' '.join(df_train[df_train['label_1']==1]['text'])
word_freq = Counter(all_text.split())
most_common = dict(word_freq.most_common(100))
least_common = dict(word_freq.most_common()[:-101:-1])
plot_wordcloud(most_common, "Самые популярные слова в первом лейбле")

Тексты, относящиеся к разным классам, содержат в большинстве предлоги и такие слова, как "билеты", "промокод", "скидка" и "ссылка"

In [ ]:
label_counts = labels.sum(axis=0)
plt.figure(figsize=(15, 6))
label_counts.plot(kind='bar')
plt.title('Распределение меток классов')
plt.xlabel('Классы')
plt.ylabel('Количество')
plt.xticks(rotation=45)
plt.show()

Здесь отчетливо виден дисбаланс классов. 2, 3, 5, 6 наименее репрезентативны. Скорее всего эти лейблы очень специфичны и возможно получиться выяснить, что это за классы

К лейблам 7, 8, 12, 13, 14, 16, 17, 18 наоборот относятся почти все тексты. Есть смысл посмотреть на те, которые не относятся

In [ ]:
class_counts = labels.sum(axis=0)
print(class_counts)

In [ ]:
phone_pattern = r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]'
phone_mask = df_train['text'].str.contains(phone_pattern, regex=True, na=False)
texts_with_phones = df_train[phone_mask]

In [ ]:
texts_with_phones

In [ ]:
texts_with_phones.iloc[:, 3:24].sum()

## **2) Проанализировать, какие очистки текста необходимы под разные способы**

1.   tf-idf + любая нейронная сеть;
2.   w2v + любая нейронная сеть;
3.   встроенный эмбеддинг в нейросетевое решение на свёрточных сетях;
4.   встроенный эмбеддинг в нейросетевое решение на рекуррентных сетях;
5.   эмбеддер + решение на bert-like моделях (любой вид).

 **Создать пайплайны очистки текста и очистить (1.5 балла).**

## **3) Поделить данные на трейн-валидацию, обучить все модели из п.2. Снабдить обучение моделей графиками отрисовки лосей и метрик, шедулерами, свитч лосей (метрик), сохранение лучшей модели, ранней остановкой, вормапом. (5 баллов).**

### ***Визуализация метрик***

In [ ]:
class MetricsPlotter:
    def __init__(self):
        self.fig, (self.ax1, self.ax2) = plt.subplots(1, 2, figsize=(15, 5))

    def plot(self, history, model_name):
        self.ax1.clear()
        self.ax2.clear()

        # График потерь
        self.ax1.plot(history.history['loss'], label='Train Loss')
        self.ax1.plot(history.history['val_loss'], label='Validation Loss')
        self.ax1.set_title(f'{model_name} - Loss')
        self.ax1.legend()

        # График метрик
        for metric in ['f1_score', 'accuracy']:
            if metric in history.history:
                self.ax2.plot(history.history[metric], label=f'Train {metric}')
                self.ax2.plot(history.history[f'val_{metric}'], label=f'Val {metric}')
        self.ax2.set_title(f'{model_name} - Metrics')
        self.ax2.legend()

        plt.tight_layout()
        plt.pause(0.1)

### ***Шедулеры обучения***

In [ ]:
def lr_scheduler(epoch, lr):
    if epoch < 5:
        return lr  # Warmup
    return lr * tf.math.exp(-0.1)

class CustomSchedule:
    def __init__(self, warmup_steps=4000):
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(64) * tf.math.minimum(arg1, arg2)

### ***Переключение лоссов/метрик***

In [ ]:
class MetricSwitcher(tf.keras.callbacks.Callback):
    def __init__(self, switch_epoch, new_metric):
        super().__init__()
        self.switch_epoch = switch_epoch
        self.new_metric = new_metric

    def on_epoch_end(self, epoch, logs=None):
        if epoch == self.switch_epoch:
            print(f"\nSwitching primary metric to {self.new_metric}")
            self.model.compile(
                optimizer=self.model.optimizer,
                loss=self.model.loss,
                metrics=[self.new_metric]
            )

### ***Ранняя остановка и сохранение***

In [ ]:
def get_callbacks(model_name):
    return [
        EarlyStopping(monitor='val_f1_score', patience=5, mode='max', verbose=1),
        ModelCheckpoint(
            f'best_{model_name}.h5',
            monitor='val_f1_score',
            save_best_only=True,
            mode='max'
        ),
        LearningRateScheduler(lr_scheduler),
        MetricSwitcher(switch_epoch=10)
    ]

### ***Warmup***

In [ ]:
class WarmupCallback(tf.keras.callbacks.Callback):
    def __init__(self, warmup_epochs=3):
        super().__init__()
        self.warmup_epochs = warmup_epochs

    def on_epoch_begin(self, epoch, logs=None):
        if epoch < self.warmup_epochs:
            for layer in self.model.layers:
                if isinstance(layer, tf.keras.layers.BatchNormalization):
                    layer.trainable = False
        else:
            for layer in self.model.layers:
                layer.trainable = True

### ***Очистка текста***

In [ ]:
#Технический код чтобы работал морф
import inspect
if not hasattr(inspect, 'getargspec'):
    import collections
    def getargspec(func):
        sig = inspect.signature(func)
        args = []
        varargs = None
        varkw = None
        defaults = []
        for param in sig.parameters.values():
            if param.kind == param.VAR_POSITIONAL:
                varargs = param.name
            elif param.kind == param.VAR_KEYWORD:
                varkw = param.name
            else:
                args.append(param.name)
                if param.default is not param.empty:
                    defaults.append(param.default)
        return collections.namedtuple('ArgSpec', 'args varargs keywords defaults')(args, varargs, varkw, tuple(defaults) if defaults else None)
    inspect.getargspec = getargspec

In [ ]:
morph = pymorphy2.MorphAnalyzer()
stemmer = RussianStemmer()
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))

### **1) tf-idf + любая нейронная сеть**

In [ ]:
import string
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
def clean_text_tfidf(text):
    # Удаление URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Удаление пунктуации
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление чисел
    text = re.sub(r'\d+', '', text)
    # Токенизация
    tokens = word_tokenize(text, language='russian')
    # Лемматизация и удаление стоп-слов
    tokens = [morph.parse(token)[0].normal_form for token in tokens if token not in russian_stopwords and len(token) > 2]
    return ' '.join(tokens)

### **2) w2v + любая нейронная сеть**

In [ ]:
def clean_text_w2v(text):
    # Удаление URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Удаление пунктуации (кроме некоторых знаков)
    text = text.translate(str.maketrans('', '', string.punctuation.replace('.', '').replace('!', '').replace('?', '')))
    # Приведение к нижнему регистру
    text = text.lower()
    # Замена чисел на токен
    text = re.sub(r'\d+', '<NUM>', text)
    # Токенизация
    tokens = word_tokenize(text, language='russian')
    # Удаление коротких токенов
    tokens = [token for token in tokens if len(token) > 1]
    return ' '.join(tokens)

### **3) встроенный эмбеддинг в нейросетевое решение на свёрточных сетях**

In [ ]:
def clean_text_cnn(text):
    # Удаление URL (замена на токен)
    text = re.sub(r'http\S+|www\S+|https\S+', '<URL>', text, flags=re.MULTILINE)
    # Удаление HTML тегов
    text = re.sub(r'<[^>]+>', '', text)
    # Удаление пунктуации
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Приведение к нижнему регистру
    text = text.lower()
    # Замена эмодзи
    text = re.sub(r'[\U0001F600-\U0001F64F]', '<EMOJI>', text)
    # Замена чисел
    text = re.sub(r'\d+', '<NUM>', text)
    return text

### **4) встроенный эмбеддинг в нейросетевое решение на рекуррентных сетях**

In [ ]:
def clean_text_rnn(text):
    # Удаление URL (замена на токен)
    text = re.sub(r'http\S+|www\S+|https\S+', '<URL>', text, flags=re.MULTILINE)
    # Удаление HTML тегов
    text = re.sub(r'<[^>]+>', '', text)
    # Сохранение некоторых знаков препинания
    text = text.translate(str.maketrans('', '', string.punctuation.replace('.', '').replace('!', '').replace('?', '').replace(',', '')))
    # Приведение к нижнему регистру
    text = text.lower()
    # Замена эмодзи
    text = re.sub(r'[\U0001F600-\U0001F64F]', '<EMOJI>', text)
    # Замена чисел
    text = re.sub(r'\d+', '<NUM>', text)
    return text


### **5) эмбеддер + решение на bert-like моделях (любой вид)**

In [ ]:
def clean_text_bert(text):
    # Минимальная очистка для BERT
    # Удаление лишних пробелов
    text = ' '.join(text.split())
    return text

In [ ]:
# Применение очистки
df_train['text_tfidf'] = df_train['text'].apply(clean_text_tfidf)
df_train['text_w2v'] = df_train['text'].apply(clean_text_w2v)
df_train['text_cnn'] = df_train['text'].apply(clean_text_cnn)
df_train['text_rnn'] = df_train['text'].apply(clean_text_rnn)
df_train['text_bert'] = df_train['text'].apply(clean_text_bert)

In [ ]:
X_test_tfidf= test['text'].apply(clean_text_tfidf)
X_test_w2v= test['text'].apply(clean_text_w2v)
X_test_cnn= test['text'].apply(clean_text_cnn)
X_test_rnn= test['text'].apply(clean_text_rnn)
X_test_bert= test['text'].apply(clean_text_bert)
test_ids = test['id']

In [ ]:
X_test_w2v

### **Поделить данные на трейн-валидацию**

In [ ]:
# Разделение на признаки и метки
labels = df_train.filter(regex='label_').values
texts_tfidf = df_train['text_tfidf'].values
texts_w2v = df_train['text_w2v'].values
texts_cnn = df_train['text_cnn'].values
texts_rnn = df_train['text_rnn'].values
texts_bert = df_train['text_bert'].values

# Разделение на train/validation
(X_train_tfidf, X_val_tfidf, y_train, y_val) = train_test_split(texts_tfidf, labels, test_size=0.2, random_state=42)
(X_train_w2v, X_val_w2v, _, _) = train_test_split(texts_w2v, labels, test_size=0.2, random_state=42)
(X_train_cnn, X_val_cnn, _, _) = train_test_split(texts_cnn, labels, test_size=0.2, random_state=42)
(X_train_rnn, X_val_rnn, _, _) = train_test_split(texts_rnn, labels, test_size=0.2, random_state=42)
(X_train_bert, X_val_bert, _, _) = train_test_split(texts_bert, labels, test_size=0.2, random_state=42)

In [ ]:
texts_w2v = df_train['text_w2v'].values
texts_w2v

1. tf-idf + любая нейронная сеть (скор на кагле)

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train_tfidf).toarray()
X_val_tfidf = tfidf.transform(X_val_tfidf).toarray()

In [ ]:
 model = Sequential([
     Dense(512, activation='relu', input_shape=(X_train_tfidf.shape[1],)),
     Dropout(0.5),
     Dense(256, activation='relu'),
     Dropout(0.5),
     Dense(y_train.shape[1], activation='sigmoid')
 ])


 model.compile(
     optimizer=Adam(learning_rate=0.001),
     loss='binary_crossentropy',
     metrics=['accuracy']
 )

 history = model.fit(
     X_train_tfidf, y_train,
     validation_data=(X_val_tfidf, y_val),
     epochs=10,
     batch_size=32,
     verbose=1
 )


 y_pred = model.predict(X_val_tfidf)
 y_pred_binary = (y_pred > 0.5).astype(int)


 print(classification_report(y_val, y_pred_binary, target_names=[f'label_{i}' for i in range(y_train.shape[1])]))

In [ ]:
X_test_tfidf = tfidf.transform(X_test_tfidf).toarray()


y_test_pred = model.predict(X_test_tfidf)
y_test_pred_binary = (y_test_pred > 0.5).astype(int)


labels = [' '.join(map(str, row)) for row in y_test_pred_binary]


results = pd.DataFrame({
     'id': test_ids,
     'labels': labels
 }).sort_values('id')

In [ ]:
ss = pd.read_csv('/content/drive/MyDrive/DL/comp2/sample_submission.csv')
labels_as_string =[]
for row in y_test_pred_binary:
  str_row = [str(label) for label in row]
  labels_as_string.append(' '.join(str_row))
ss['labels'] = labels_as_string
ss.to_csv('sample_submission.csv', index = False)

In [ ]:
ss

2.  w2v + любая нейронная сеть

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Conv1D, GlobalMaxPooling1D,LSTM

In [ ]:
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Обучение Word2Vec модели
tokenized_texts = [text.split() for text in X_train]
w2v_model = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

# Создание матрицы эмбеддингов
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1

embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

# Преобразование текстов в последовательности
X_train_seq = tokenizer.texts_to_sequences(X_train_w2v)
X_val_seq = tokenizer.texts_to_sequences(X_val_w2v)
X_test_seq = tokenizer.texts_to_sequences(test['text'])

max_len = 100
X_train_w2v = pad_sequences(X_train_seq, maxlen=max_len)
X_val_w2v = pad_sequences(X_val_seq, maxlen=max_len)
X_test_w2v = pad_sequences(X_test_seq, maxlen=max_len)

# Модель
model = Sequential([
    Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='sigmoid')
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Обучение
history = model.fit(
    X_train_w2v, y_train,
    validation_data=(X_val_w2v, y_val),
    epochs=10,
    batch_size=128,
    verbose=1
)

# Оценка
y_pred = model.predict(X_val_w2v)
y_pred_binary = (y_pred > 0.5).astype(int)
print("Word2Vec + NN Classification Report:")
print(classification_report(y_val, y_pred_binary, target_names=[f'label_{i}' for i in range(y_train.shape[1])]))

In [ ]:
y_test_pred = model.predict(X_test_w2v)
y_test_pred_binary = (y_test_pred > 0.5).astype(int)


labels = [' '.join(map(str, row)) for row in y_test_pred_binary]


results = pd.DataFrame({
     'id': test_ids,
     'labels': labels
 }).sort_values('id')

In [ ]:
ss = pd.read_csv('/content/drive/MyDrive/DL/comp2/sample_submission.csv')
labels_as_string =[]
for row in y_test_pred_binary:
  str_row = [str(label) for label in row]
  labels_as_string.append(' '.join(str_row))
ss['labels'] = labels_as_string
ss.to_csv('sample_submission.csv', index = False)

3.

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

vocab_size = len(tokenizer.word_index) + 1
max_len = 100

# 2. Преобразование текстов в последовательности
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(test['text'])

# 3. Добавление паддинга
X_train_cnn = pad_sequences(X_train_seq, maxlen=max_len)
X_val_cnn = pad_sequences(X_val_seq, maxlen=max_len)
X_test_cnn = pad_sequences(X_test_seq, maxlen=max_len)

# 4. Создание модели CNN
model = Sequential([
    Embedding(
        input_dim=vocab_size,
        output_dim=100,
        input_length=max_len
    ),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='sigmoid')  # y_train - one-hot encoded labels
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 5. Обучение модели
history = model.fit(
    X_train_cnn, y_train,
    validation_data=(X_val_cnn, y_val),
    epochs=10,
    batch_size=128,
    verbose=1
)

# 6. Оценка модели
y_pred = model.predict(X_val_cnn)
y_pred_binary = (y_pred > 0.5).astype(int)
print("CNN Classification Report:")
print(classification_report(y_val, y_pred_binary, target_names=[f'label_{i}' for i in range(y_train.shape[1])]))

4.

In [ ]:
model = Sequential([
    Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_len, trainable=False),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='sigmoid')
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train_w2v, y_train,
    validation_data=(X_val_w2v, y_val),
    epochs=10,
    batch_size=32,
    verbose=1
)

y_pred = model.predict(X_val_w2v)
y_pred_binary = (y_pred > 0.5).astype(int)
print("RNN (LSTM) Classification Report:")
print(classification_report(y_val, y_pred_binary, target_names=[f'label_{i}' for i in range(y_train.shape[1])]))

5.

In [ ]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

# Загрузка BERT модели и токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')

# Токенизация
X_train_bert = tokenizer(X_train.tolist(), padding=True, truncation=True, max_length=100, return_tensors="tf")
X_val_bert = tokenizer(X_val.tolist(), padding=True, truncation=True, max_length=100, return_tensors="tf")
X_test_bert = tokenizer(X_test.tolist(), padding=True, truncation=True, max_length=100, return_tensors="tf")

# Создание модели
input_ids = tf.keras.layers.Input(shape=(100,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(100,), dtype=tf.int32, name="attention_mask")

bert_output = bert_model(input_ids, attention_mask=attention_mask)[1]
dense1 = tf.keras.layers.Dense(256, activation='relu')(bert_output)
dropout = tf.keras.layers.Dropout(0.5)(dense1)
output = tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid')(dropout)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

model.compile(
    optimizer=Adam(learning_rate=2e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Обучение
history = model.fit(
    {'input_ids': X_train_bert['input_ids'], 'attention_mask': X_train_bert['attention_mask']},
    y_train,
    validation_data=({'input_ids': X_val_bert['input_ids'], 'attention_mask': X_val_bert['attention_mask']}, y_val),
    epochs=3,  # BERT требует меньше эпох
    batch_size=16,
    verbose=1
)

# Оценка
y_pred = model.predict({'input_ids': X_val_bert['input_ids'], 'attention_mask': X_val_bert['attention_mask']})
y_pred_binary = (y_pred > 0.5).astype(int)
print("BERT Classification Report:")
print(classification_report(y_val, y_pred_binary, target_names=[f'label_{i}' for i in range(y_train.shape[1])]))

In [ ]:
# X_test_clean = test['text'].apply(basic_clean)
# test_ids = test['id']

# X_test_tfidf = tfidf.transform(X_test_clean).toarray()


# y_test_pred = model.predict(X_test_tfidf)
# y_test_pred_binary = (y_test_pred > 0.5).astype(int)


# labels = [' '.join(map(str, row)) for row in y_test_pred_binary]


# results = pd.DataFrame({
#     'id': test_ids,
#     'labels': labels
# }).sort_values('id')

In [ ]:
# X_test_clean = test['text'].apply(basic_clean)
# test_ids = test['id']

# X_test_tfidf = tfidf.transform(X_test_clean).toarray()


# y_test_pred = model.predict(X_test_tfidf)
# y_test_pred_binary = (y_test_pred > 0.5).astype(int)


# labels = [' '.join(map(str, row)) for row in y_test_pred_binary]


# results = pd.DataFrame({
#     'id': test_ids,
#     'labels': labels
# }).sort_values('id')
